In [18]:
import numpy as np
from numpy import linalg as LA
import time, random
import matplotlib.pyplot as plt
import math
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score,train_test_split

In [19]:
knot_info = '/Users/guoyihan/Downloads/knotinfo_data_complete.xls'

In [20]:
Knots = pd.read_excel(knot_info, 
skiprows = [1],
header=0,
index_col=False,
keep_default_na=True)

In [21]:
Knots

,name,name_anon,diagram,diagram_anon,category,category_anon,knot_atlas,knot_atlas_anon,knotilus_page,knotilus_page_anon,...,adequate,adequate_anon,montesinos_notation,montesinos_notation_anon,boundary_slopes,boundary_slopes_anon,pretzel_notation,pretzel_notation_anon,double_slice_genus,double_slice_genus_anon
0,0_1,diagram_display.php?_1,0_1-50.png,diagram_display.php?_1,0,NaN,0.1,http://katlas.math.toronto.edu/wiki/0_1,NaN,NaN,...,Y,NaN,NaN,NaN,NaN,NaN,P(1),NaN,NaN,NaN
1,3_1,diagram_display.php?3_1,3_1-50.png,diagram_display.php?3_1,3,NaN,3.1,http://katlas.math.toronto.edu/wiki/3_1,Go,http://srankin.math.uwo.ca/cgi-bin/retrieve.cg...,...,Y,NaN,K(1/3),NaN,"{0,6}",NaN,"P(-1,-1,-1)",NaN,2,NaN
2,4_1,diagram_display.php?4_1,4_1-50.png,diagram_display.php?4_1,4,NaN,4.1,http://katlas.math.toronto.edu/wiki/4_1,Go,http://srankin.math.uwo.ca/cgi-bin/retrieve.cg...,...,Y,NaN,K(2/5),NaN,"{-4,0,4}",NaN,"P(1,1,2)",NaN,2,NaN
3,5_1,diagram_display.php?5_1,5_1-50.png,diagram_display.php?5_1,5,NaN,5.1,http://katlas.math.toronto.edu/wiki/5_1,Go,http://srankin.math.uwo.ca/cgi-bin/retrieve.cg...,...,Y,NaN,K(1/5),NaN,"{0,10}",NaN,"P(-1,-1,-1,-1,-1)",NaN,4,NaN
4,5_2,diagram_display.php?5_2,5_2-50.png,diagram_display.php?5_2,5,NaN,5.2,http://katlas.math.toronto.edu/wiki/5_2,Go,http://srankin.math.uwo.ca/cgi-bin/retrieve.cg...,...,Y,NaN,K(3/7),NaN,"{0,4,10}",NaN,"P(-1,-1,-3)",NaN,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2973,12n_884,diagram_display.php?12n_884,12n_884-50.png,diagram_display.php?12n_884,12n,NaN,NaN,NaN,Go,http://srankin.math.uwo.ca/cgi-bin/retrieve.cg...,...,NaN,NaN,Not Montesinos,NaN,NaN,NaN,NaN,NaN,2,NaN
2974,12n_885,diagram_display.php?12n_885,12n_885-50.png,diagram_display.php?12n_885,12n,NaN,NaN,NaN,Go,http://srankin.math.uwo.ca/cgi-bin/retrieve.cg...,...,NaN,NaN,Not Montesinos,NaN,NaN,NaN,NaN,NaN,2,NaN
2975,12n_886,diagram_display.php?12n_886,12n_886-50.png,diagram_display.php?12n_886,12n,NaN,NaN,NaN,Go,http://srankin.math.uwo.ca/cgi-bin/retrieve.cg...,...,NaN,NaN,Not Montesinos,NaN,NaN,NaN,NaN,NaN,2,NaN
2976,12n_887,diagram_display.php?12n_887,12n_887-50.png,diagram_display.php?12n_887,12n,NaN,NaN,NaN,Go,http://srankin.math.uwo.ca/cgi-bin/retrieve.cg...,...,NaN,NaN,Not Montesinos,NaN,NaN,NaN,NaN,NaN,4,NaN


In [22]:
Y = Knots['alternating']

In [375]:
X = Knots[['determinant',
           'crossing_number',
           'unknotting_number',
           'three_genus',
           'bridge_index',
           'braid_index',
           'dt_rank',
           ]]

In [376]:
X

,determinant,crossing_number,unknotting_number,three_genus,bridge_index,braid_index,dt_rank
0,0,0,0,0,1,1,1
1,3,3,1,1,2,2,2
2,5,4,1,1,2,3,3
3,5,5,2,2,2,2,5
4,7,5,1,1,2,3,4
...,...,...,...,...,...,...,...
2973,81,12,1,2,3,5,2974
2974,149,12,-1,3,3,5,2975
2975,159,12,2,3,3,5,2976
2976,125,12,2,4,3,3,2977


In [377]:
def convert_list_to_int(unknotting):
    try:
        return int(unknotting)
    except:
        return -1
    
convert_list_to_int_vect = np.vectorize(convert_list_to_int)

In [378]:
Knots['unknotting_number'] = convert_list_to_int_vect(Knots['unknotting_number'])

In [379]:
mask = Knots['unknotting_number'] >= 0
X = X[mask]

In [380]:
def YN_to_int(let):
    if let == 'Y':
        return 1
    elif let == 'N':
        return 0
    else:
        print(let)
        
YN_to_int_vect = np.vectorize(YN_to_int)

In [381]:
yp = YN_to_int_vect(Y)

yp = yp[mask]

In [382]:
x_train, x_test, y_train, y_test = train_test_split(X, yp, random_state=0)

In [383]:
logisticRegr = LogisticRegression(max_iter = 10000)

In [384]:
clf = logisticRegr.fit(x_train,y_train)

In [385]:
clf.score(x_train,y_train)

0.9671848013816926

In [386]:
def cross_value(cross):
    A = []
    B = []
    for i in range(2,cross):
        a = cross_val_score(clf, x_train, y_train, cv= i)
        b = cross_val_score(clf, x_train, y_train, cv=i).mean()
        A.append(a)
        B.append(b)
    for i in range(len(B)):
        if B[i] == max(B):
            return max(B),A[i]

In [387]:
cross_value(10)

(0.9671848013816926, array([0.96891192, 0.96545769, 0.9671848 ]))

In [388]:
logisticRegr.predict(x_test)

array([1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,

In [389]:
y_test

array([0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1,

In [390]:
# variables below are not using becuase it contains a list or Nah which can not be converted to int
#'topological_four_genus','smooth_concordance_genus', 'crosscap_number','rasmussen_invariant',
#'ozsvath_szabo_tau_invariant','arf_invariant'

('ozsvath_szabo_tau_invariant', 'arf_invariant')